In [23]:
import numpy as np
import random
import pandas as pd
import cv2

In [24]:
# Define empty patches set
empty = []

In [25]:
# Read file with sea lions patches coordinates
sealions_df = pd.read_csv('./sealions.csv')
file_names = sealions_df.filename.unique()

In [26]:
# Loop on each file
for file in file_names:
    # Select rows from data frame with same name file
    df = sealions_df[sealions_df['filename'] == file]
    # Get image size
    image = cv2.imread("./kaggle_sea_lions/Train/" + file)
    IM_SIZE_Y, IM_SIZE_X, _ = image.shape
    # Create array for coordinates to exclude
    exclude = []
    # Create array for empty patches
    empty_patches_coord = []
    # Loop on each row of the specified file
    for row in df.iterrows():
        # Select coordinates for patches where sea lions are present
        p_1 = ((((int(row[1]['coord_x']) + 48) // 96) * 96) + 48, (((int(row[1]['coord_y']) + 48) // 96) * 96) + 48)
        p_2 = ((((int(row[1]['coord_x']) + 48) // 96) * 96) + 48, (((int(row[1]['coord_y']) - 48) // 96) * 96) + 48)
        p_3 = ((((int(row[1]['coord_x']) - 48) // 96) * 96) + 48, (((int(row[1]['coord_y']) + 48) // 96) * 96) + 48)
        p_4 = ((((int(row[1]['coord_x']) - 48) // 96) * 96) + 48, (((int(row[1]['coord_y']) - 48) // 96) * 96) + 48)
        # Add coordinates to list
        exclude.append(p_1)
        exclude.append(p_2)
        exclude.append(p_3)
        exclude.append(p_4)
    # Generate coordinates of all square patches 96x96 such that they don't intersect between themselves
    for i in range(48, IM_SIZE_X - 48, 96):
        for j in range(48, IM_SIZE_Y - 48, 96):
            empty_patches_coord.append((i,j))
    # Keep patches that does not contain sea lions with sets difference
    empty_patches_coord = list(set(empty_patches_coord) - set(exclude))
    for coord in empty_patches_coord:
        empty.append(list((int(coord[0]), int(coord[1]), file)))

In [27]:
# Take a number of empty patches equal to the number of sea lions
n = len(sealions_df.index)
empty = random.sample(empty, n)

In [28]:
empty_df = pd.DataFrame(data=empty, columns=["coord_x", "coord_y", "filename"])
empty_df.to_csv('./empty.csv', index=False)